In [1]:
#!pip install -q git+https://github.com/deepmipt/bert.git@feat/keras1 tensorflow-gpu==1.14.0rc0 deeppavlov

In [2]:
import tensorflow as tf

In [3]:
tf.__version__

'1.14.0-rc0'

In [4]:
# tf.compat.v1.enable_eager_execution()

In [5]:
tf.compat.v1.executing_eagerly()

False

# Paraphrases data

In [6]:
BATCH_SIZE = 64  # both for training and evaluation for now
MAX_LEN = 52  # the longest sequence in the paraphrase dataset, workaroud that make shape of all batch tensors equal
NUM_TRAIN_SAMPLES = 6656  # total of 7202 samples minus ~500 (actually 546) samples for validation

In [7]:
tf.keras.utils.get_file(
    fname='paraphraser.zip',
    origin='http://files.deeppavlov.ai/datasets/paraphraser.zip',
    cache_subdir='dataset',
    extract=True,
    cache_dir='.'
)

'./dataset/paraphraser.zip'

In [8]:
tf.keras.utils.get_file(
    fname='paraphraser_gold.zip',
    origin='http://files.deeppavlov.ai/datasets/paraphraser_gold.zip',
    cache_subdir='dataset',
    extract=True,
    archive_format='zip',
    cache_dir='.'
)

'./dataset/paraphraser_gold.zip'

In [9]:
tf.keras.utils.get_file(
    fname='rubert_cased_L-12_H-768_A-12_v1_converted.zip',
    origin='http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_v1_converted.zip',
    cache_subdir='models',
    extract=True,
    cache_dir='.'
)

'./models/rubert_cased_L-12_H-768_A-12_v1_converted.zip'

In [7]:
def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()


def data_gen(train_data=True):
    from deeppavlov.dataset_readers.paraphraser_reader import ParaphraserReader
    from bert_dp.tokenization import FullTokenizer
    ds = ParaphraserReader().read(data_path='dataset', do_lower_case=False)
    tokenizer = FullTokenizer(vocab_file='models/rubert_cased_L-12_H-768_A-12_v1/vocab.txt', do_lower_case=False)
    tds = ds['train'] if train_data else ds['test']
    for pair, label in tds:
        t1 = tokenizer.tokenize(pair[0])
        t2 = tokenizer.tokenize(pair[1])
        _truncate_seq_pair(t1, t2, 512)
        s = ['[CLS]'] + t1 + ['[SEP]'] + t2 + ['[SEP]']
        yield tokenizer.convert_tokens_to_ids(s), label  # tf.one_hot(label, depth=2, dtype=tf.int32)

In [8]:
train_ds_ = tf.data.Dataset.from_generator(data_gen,
                                           output_types=(tf.int32, tf.int32),
                                           output_shapes=(tf.TensorShape([None]), tf.TensorShape([])))

W0605 16:41:17.668468 139940610430784 deprecation.py:323] From /home/nab/PycharmProjects/work/venv/lib/python3.6/site-packages/tensorflow/python/data/ops/dataset_ops.py:494: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


In [9]:
train_ds = train_ds_.take(NUM_TRAIN_SAMPLES)
valid_ds = train_ds_.skip(NUM_TRAIN_SAMPLES)
batched_train_ds = train_ds.padded_batch(batch_size=BATCH_SIZE,
                                         padded_shapes=(MAX_LEN,  # tf.constant([-1], dtype=tf.int64),
                                                        ()),  # 2),
                                         drop_remainder=True)  # fixed batch size for now
batched_valid_ds = valid_ds.padded_batch(batch_size=BATCH_SIZE,
                                         padded_shapes=(MAX_LEN,  # tf.constant([-1], dtype=tf.int64),
                                                        ()),  # 2),
                                         drop_remainder=True)
test_ds = tf.data.Dataset.from_generator(data_gen,
                                         output_types=(tf.int32, tf.int32),
                                         output_shapes=(tf.TensorShape([None]), tf.TensorShape([])),
                                         args=(False,))
batched_test_ds = test_ds.padded_batch(batch_size=BATCH_SIZE,
                                       padded_shapes=(MAX_LEN,  # tf.constant([-1], dtype=tf.int64),
                                                      ()),  # 2),
                                       drop_remainder=True)  # fixed batch size for now

# Model construction and compilation for training

In [10]:
from bert_dp import bert
# some data to pass through model in order to construct and autograph it
bert_layer = bert.BERT(name='bert') # naming is important for compatibility with checkpoints
for s in batched_train_ds:
    r = bert_layer(s[0])
    break

W0605 16:41:22.793537 139940610430784 deprecation_wrapper.py:119] From /home/nab/PycharmProjects/work/bert/bert_dp/embeddings.py:30: The name tf.keras.initializers.TruncatedNormal is deprecated. Please use tf.compat.v1.keras.initializers.TruncatedNormal instead.

W0605 16:41:22.794115 139940610430784 deprecation.py:506] From /home/nab/PycharmProjects/work/venv/lib/python3.6/site-packages/tensorflow/python/keras/initializers.py:94: calling TruncatedNormal.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0605 16:41:22.799962 139940610430784 deprecation.py:506] From /home/nab/PycharmProjects/work/venv/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instru

In [11]:
try:
    del paraphrase_detector
except NameError:
    pass
paraphrase_detector = tf.keras.Sequential([bert_layer, tf.keras.layers.Dense(1, activation='sigmoid')])

In [13]:
from bert_dp.metrics import BinaryF1Score

NUM_EPOCHS = 10
VAL_EVERY_N_BATCHES = 52  # ~50, but better suited for paraphraser dataset
TRAIN_STEPS_PER_EPOCH = NUM_TRAIN_SAMPLES // BATCH_SIZE
VALIDATION_STEPS = 8  # calculated by hand for now (~546 samples in batches of 64 with droppping remainder)
LEARNING_RATE = 2e-05
# learning_rate = tf.keras.optimizers.schedules.PolynomialDecay(initial_learning_rate=LEARNING_RATE,
#                                                               # n. of batches per epoch * ~NUM_EPOCHS for convergence 
#                                                               decay_steps=STEPS_PER_EPOCH * NUM_EPOCHS,
#                                                               end_learning_rate=1e-06)
# from bert_dp.weight_decay_optimizers import AdamW
paraphrase_detector.compile(#optimizer=AdamW(weight_decay=0.01,
#                                             learning_rate=LEARNING_RATE,
#                                             epsilon=1e-6),  # as in bert_dp
                            optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE,
                                                               epsilon=1e-6),
                            loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                            metrics=[tf.keras.metrics.BinaryAccuracy(),
#                                      tf.keras.metrics.Precision(class_id=1),
#                                      tf.keras.metrics.Recall(class_id=1),
                                     tf.keras.metrics.Precision(),
                                     tf.keras.metrics.Recall(),
                                     BinaryF1Score()
                                    ])

# Training

In [14]:
paraphrase_detector.evaluate(batched_test_ds)

W0605 16:41:47.683248 139940610430784 deprecation.py:323] From /home/nab/PycharmProjects/work/venv/lib/python3.6/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0605 16:41:50.440803 139934998292224 deprecation_wrapper.py:119] From /home/nab/PycharmProjects/work/bert/bert_dp/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



     29/Unknown - 7s 236ms/step - loss: 0.7108 - binary_accuracy: 0.4957 - precision: 0.6295 - recall: 0.3811 - binary_f1score: 0.4747

In [17]:
bert_layer.load_weights('models/rubert_cased_L-12_H-768_A-12_v1_converted/bert_model').assert_consumed()

In [18]:
paraphrase_detector.evaluate(batched_test_ds)

     29/Unknown - 6s 219ms/step - loss: 0.7059 - binary_accuracy: 0.4952 - precision: 0.5973 - recall: 0.4784 - binary_f1score: 0.5313

[0.7059423676852522, 0.49515086, 0.59730035, 0.4783784, 0.5312656]

In [19]:
history = paraphrase_detector.fit(x=batched_train_ds.repeat(),
                                  y=None,
                                  epochs=NUM_EPOCHS * TRAIN_STEPS_PER_EPOCH // VAL_EVERY_N_BATCHES,  # workaround for more frequent evaluation
                                  callbacks=[tf.keras.callbacks.TensorBoard(batch_size=BATCH_SIZE),
                                             tf.keras.callbacks.ReduceLROnPlateau(monitor='val_binary_f1score',
                                                                                  factor=0.5,
                                                                                  patience=3,
                                                                                  min_lr=1e-06)],
                                  validation_data=batched_valid_ds,
                                  steps_per_epoch=VAL_EVERY_N_BATCHES,
                                  validation_steps=VALIDATION_STEPS)

Epoch 1/20
52/52 [==============================] - 30s 582ms/step - loss: 0.7495 - binary_accuracy: 0.6142 - precision: 0.6693 - recall: 0.8443 - binary_f1score: 0.7466 - val_loss: 0.6736 - val_binary_accuracy: 0.6133 - val_precision: 0.6133 - val_recall: 1.0000 - val_binary_f1score: 0.7603
Epoch 2/20
52/52 [==============================] - 24s 459ms/step - loss: 0.6813 - binary_accuracy: 0.6073 - precision: 0.6303 - recall: 0.8877 - binary_f1score: 0.7372 - val_loss: 0.6673 - val_binary_accuracy: 0.6133 - val_precision: 0.6133 - val_recall: 1.0000 - val_binary_f1score: 0.7603
Epoch 3/20
52/52 [==============================] - 24s 468ms/step - loss: 0.6594 - binary_accuracy: 0.6605 - precision: 0.6743 - recall: 0.9589 - binary_f1score: 0.7918 - val_loss: 0.6814 - val_binary_accuracy: 0.6133 - val_precision: 0.6133 - val_recall: 1.0000 - val_binary_f1score: 0.7603
Epoch 4/20
52/52 [==============================] - 25s 481ms/step - loss: 0.6763 - binary_accuracy: 0.6058 - precision: 